# Financiamiento público ajustado a la inflación a precios constantes:

## Cargamos las Librerías:

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import janitor
import numpy as np
import pandas as pd
import session_info
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

/home/barea/anaconda3/envs/analisis_exploratorio/lib/python3.9/site-packages/janitor/utils.py:84: FutureWarning: PandasArray has been renamed NumpyExtensionArray. Use that instead. This alias will be removed in a future version.
  @_expand_grid.register(pd.arrays.PandasArray)


## Establecemos los Parámetros de los Gráficos:

In [2]:
%matplotlib inline
sns.set_style(style='whitegrid')
sns.set_context(context='notebook')
plt.rcParams['figure.figsize'] = (11, 9.4)

## Cargamos los Conjuntos de Datos:

In [3]:
inpc_data = pd.read_excel('~/inflacion/data/IPC.xlsx')
inpc_data

,Fecha,índices
0,Ene 1997,29.50
1,Feb 1997,29.99
2,Mar 1997,30.37
3,Abr 1997,30.70
4,May 1997,30.98
...,...,...
319,Ago 2023,129.54
320,Sep 2023,130.12
321,Oct 2023,130.61
322,Nov 2023,131.44


In [4]:
financiamiento = pd.read_csv('~/inflacion/data/financiamiento_historico.csv')
financiamiento

,Ambito,Año,Partido,Actividades Ordinarias,Gastos de Campaña,Actividades Específicas,Total,Unnamed: 7
0,Federal,1997,Partido Acción Nacional,259956828.81,259956828.81,7334453.45,527248111.07,NaN
1,Federal,1997,Partido Revolucionario Institucional,437011758.76,437011758.76,18089139.75,892112657.27,NaN
2,Federal,1997,Partido de la Revolución Democrática,194531523.78,194531523.78,2272992.90,391336040.46,NaN
3,Federal,1997,Partido del Trabajo,92994946.66,92994946.66,3947624.70,189937518.02,NaN
4,Federal,1997,Partido Cardenista,15751920.92,19689901.16,2382957.21,37824779.29,NaN
...,...,...,...,...,...,...,...,...
156,Federal,2013,Partido de la Revolución Democrática,634867508.95,NaN,19046025.27,653913534.22,NaN
157,Federal,2013,Partido del Trabajo,273435553.55,NaN,8203066.61,281638620.16,NaN
158,Federal,2013,Partido Verde Ecologista de México,313466657.34,NaN,9403999.72,322870657.06,NaN
159,Federal,2013,Movimiento Ciudadano,257877302.28,NaN,7736319.07,265613621.35,NaN


In [5]:
financiamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ambito                   161 non-null    object 
 1   Año                      161 non-null    int64  
 2   Partido                  161 non-null    object 
 3   Actividades Ordinarias   145 non-null    float64
 4   Gastos de Campaña        83 non-null     float64
 5   Actividades Específicas  147 non-null    float64
 6   Total                    161 non-null    float64
 7   Unnamed: 7               0 non-null      float64
dtypes: float64(5), int64(1), object(2)
memory usage: 10.2+ KB


## Realizamos una Agrupación para Saber la Inflación de las Cifras:

In [6]:
financiamiento_historico = financiamiento.groupby('Año')['Total'].sum().reset_index(name = 'Financiamiento')
financiamiento_historico

,Año,Financiamiento
0,1997,2446723214.77
1,1998,1046566954.67
2,1999,1312364397.62
3,2000,3064092232.97
4,2001,2250685592.84
5,2002,2439871186.19
6,2003,4942596068.79
7,2004,1854015953.66
8,2005,2013252441.28
9,2006,4171096908.42


In [7]:
financiamiento_historico['Financiamiento'].sum()

49590966508.71999

In [8]:
financiamiento_dict = financiamiento_historico.set_index('Año')['Financiamiento'].to_dict()
financiamiento_dict

{1997: 2446723214.77,
 1998: 1046566954.67,
 1999: 1312364397.62,
 2000: 3064092232.9700003,
 2001: 2250685592.84,
 2002: 2439871186.19,
 2003: 4942596068.79,
 2004: 1854015953.66,
 2005: 2013252441.28,
 2006: 4171096908.42,
 2007: 2704100762.62,
 2008: 2690311483.58,
 2009: 3631639027.92,
 2010: 2997358834.02,
 2011: 3212932808.88,
 2012: 5142514887.61,
 2013: 3670843752.88}

## Aplicamos el Script para Conocer la Inflación:

### Convertimos las Fechas en un Formato Utilizable:

In [9]:
meses_espanol = {
    "Ene": "Jan", "Feb": "Feb", "Mar": "Mar", "Abr": "Apr", "May": "May", "Jun": "Jun",
    "Jul": "Jul", "Ago": "Aug", "Sep": "Sep", "Oct": "Oct", "Nov": "Nov", "Dic": "Dec"
}

def traducir_fecha(fecha):
    mes, año = fecha.split()
    return f"{meses_espanol[mes]} {año}"


inpc_data['Fecha'] = inpc_data['Fecha'].apply(traducir_fecha)

inpc_data['Fecha'] = pd.to_datetime(inpc_data['Fecha'], format='%b %Y')

inpc_data['Año'] = inpc_data['Fecha'].dt.year

In [10]:
inpc_data

,Fecha,índices,Año
0,1997-01-01,29.50,1997
1,1997-02-01,29.99,1997
2,1997-03-01,30.37,1997
3,1997-04-01,30.70,1997
4,1997-05-01,30.98,1997
...,...,...,...
319,2023-08-01,129.54,2023
320,2023-09-01,130.12,2023
321,2023-10-01,130.61,2023
322,2023-11-01,131.44,2023


### Calculamos el Promedio del INPC:

In [11]:
promedio_inpc_anual = inpc_data.groupby('Año')['índices'].mean()
promedio_inpc_anual

Año
1997    31.41
1998    36.41
1999    42.45
2000    46.48
2001    49.43
2002    51.92
2003    54.28
2004    56.83
2005    59.09
2006    61.24
2007    63.67
2008    66.93
2009    70.48
2010    73.41
2011    75.91
2012    79.03
2013    82.04
2014    85.33
2015    87.65
2016    90.13
2017    95.57
2018   100.26
2019   103.90
2020   107.43
2021   113.54
2022   122.51
2023   129.28
Name: índices, dtype: float64

### Ajustamos las Cifras al Valor Actual Usando el Promedio Anual del INPC

In [12]:
ipc_actual = promedio_inpc_anual[2023]

### Conocemos los Resultados:

In [13]:
financiamiento_ajustado = {}

for año, monto in financiamiento_dict.items():
    if año in promedio_inpc_anual.index: 
        ipc_promedio_año = promedio_inpc_anual[año]
        financiamiento_ajustado[año] = monto * (ipc_actual / ipc_promedio_año)
    else:
        print(f"Año {año} no encontrado en los datos de promedio INPC. Omisión del cálculo.")

print("Cifras ajustadas al valor actual (2023):")
for año, ajuste in financiamiento_ajustado.items():
    print(f"Año {año}: ${ajuste:.2f}")

Cifras ajustadas al valor actual (2023):
Año 1997: $10071809553.09
Año 1998: $3716206742.64
Año 1999: $3997075076.98
Año 2000: $8523325148.92
Año 2001: $5885889911.99
Año 2002: $6075021338.67
Año 2003: $11771312185.67
Año 2004: $4217786896.37
Año 2005: $4404391558.29
Año 2006: $8805514136.14
Año 2007: $5490759540.78
Año 2008: $5196443222.91
Año 2009: $6661757581.02
Año 2010: $5278828597.71
Año 2011: $5472035833.82
Año 2012: $8412481219.12
Año 2013: $5784826819.88


## Convertimos el resultado a Dataframe:

In [14]:
df_financiamiento_ajustado = pd.DataFrame(
    list(financiamiento_ajustado.items()), 
    columns=['Año', 'Financiamiento Ajustado']
)

df_financiamiento_ajustado = df_financiamiento_ajustado.sort_values(by='Año')
df_financiamiento_ajustado

,Año,Financiamiento Ajustado
0,1997,10071809553.09
1,1998,3716206742.64
2,1999,3997075076.98
3,2000,8523325148.92
4,2001,5885889911.99
5,2002,6075021338.67
6,2003,11771312185.67
7,2004,4217786896.37
8,2005,4404391558.29
9,2006,8805514136.14


In [15]:
df_financiamiento_ajustado['Financiamiento Ajustado'].sum()

109765465363.99147

## Cargamos el Conjunto de Datos entre 2014 al 2022:

In [16]:
financiamiento_vigente = pd.read_csv('~/inflacion/data/financiamiento.csv')
financiamiento_vigente

,Año,Partido,Concepto,Categoria,Monto
0,2014,PAN,Actividades Ordinarias Permanentes,Igualitario,163319404.04
1,2014,PRI,Actividades Ordinarias Permanentes,Igualitario,163319404.04
2,2014,PRD,Actividades Ordinarias Permanentes,Igualitario,163319404.04
3,2014,PT,Actividades Ordinarias Permanentes,Igualitario,163319404.04
4,2014,PVEM,Actividades Ordinarias Permanentes,Igualitario,163319404.04
...,...,...,...,...,...
516,2023,PRD,Actividades Específicas,Proporcional,5098946.00
517,2023,PT,Actividades Específicas,Proporcional,4535748.00
518,2023,PVEM,Actividades Específicas,Proporcional,7594588.00
519,2023,MC,Actividades Específicas,Proporcional,9781786.00


### Realizamos la Agrupación:

In [17]:
financiamiento_vigente = financiamiento_vigente.groupby('Año')['Monto'].sum().reset_index(name = 'Financiamiento')
financiamiento_vigente

,Año,Financiamiento
0,2014,4002018895.98
1,2015,5122198499.05
2,2016,4031121882.70
3,2017,4138727087.00
4,2018,6745554160.00
5,2019,4965828351.00
6,2020,5239001651.00
7,2021,6871356602.00
8,2022,5821851702.00
9,2023,6233510798.00


In [18]:
financiamiento_vigente['Financiamiento'].sum()

53171169628.729996

### Creamos el Diccionario con los Datos:

In [19]:
financiamiento_dict = financiamiento_vigente.set_index('Año')['Financiamiento'].to_dict()
financiamiento_dict

{2014: 4002018895.98,
 2015: 5122198499.05,
 2016: 4031121882.7,
 2017: 4138727087.0,
 2018: 6745554160.0,
 2019: 4965828351.0,
 2020: 5239001651.0,
 2021: 6871356602.0,
 2022: 5821851702.0,
 2023: 6233510798.0}

### Conocemos los Resultados de los Datos entre 2014 al 2022:

In [20]:
financiamiento_ajustado = {
    año: monto * (ipc_actual / promedio_inpc_anual[año])
    for año, monto in financiamiento_dict.items()
    if año in promedio_inpc_anual.index
}

df_financiamiento_ajustado = pd.DataFrame(
    list(financiamiento_ajustado.items()), 
    columns=['Año', 'Financiamiento Ajustado']
).sort_values(by='Año')

print(df_financiamiento_ajustado)

    Año  Financiamiento Ajustado
0  2014            6063069149.40
1  2015            7554610346.87
2  2016            5782248888.18
3  2017            5598374665.47
4  2018            8698412574.92
5  2019            6178792249.72
6  2020            6304536787.74
7  2021            7823777483.54
8  2022            6143681386.18
9  2023            6233510798.00


In [21]:
df_financiamiento_ajustado['Financiamiento Ajustado'].sum()

66381014330.02352